In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
!ls
df = pd.read_csv('/content/drive/Shared drives/DS4A_2021/Team74/data/Stage/sampled_data_stage.csv')
df.head(3)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.dtypes

In [ ]:
df.shape

Convert Data type

In [ ]:
# df['BorrowerZip'] = df['BorrowerZip'].astype(int)
df['DateApproved'] = pd.to_datetime(df['DateApproved'])
df['LoanStatusDate'] = pd.to_datetime(df['LoanStatusDate']) 
# df['BorrowerZip'] = df['BorrowerZip'][:5]
df['BorrowerZip'] = df['BorrowerZip'].astype(str).str.split('-').str[0]
df['BorrowerZip'] = df['BorrowerZip'].str.split('.').str[0]
#
#df['BorrowerZip'] = df['BorrowerZip']
# df = df.apply (pd.to_numeric, errors='coerce')
# df = df.dropna()
#df['ServicingLenderZip'] = df['ServicingLenderZip'].str.split('-').str[0].astype(int)
df.head(10)

In [ ]:
# Only focusing on NY, specifically NYC
df = df[df['BorrowerState'] == 'NY']

In [ ]:
import os
import json

In [ ]:
# df['BorrowerZip'] = df['BorrowerZip'].apply(pd.to_numeric, errors='coerce')
# df['BorrowerZip'] = df['BorrowerZip'].dropna()


# zips = ((np.asarray(df['BorrowerZip'])))
# result = df.pivot(index = 'BorrowerName', columns = 'BorrowerAddress', values = 'BorrowerZip')

# #array to annotate heatmaps
# labels = (np.asarray(["{0} \n {1:2f}".format(value)
#                                              for value in zip(zips.flatten())]))

# fig, ax = plt.subplots(figsize = (12,7))
# title = "Heatmap by Zipcodes"
# plt.title(title, fontsize = 18)
# ttl = ax.titlettl.set_position([0.5,1.05])

# print(result)
# df.head(3)
# Set zipcode type to string (folium)
df['BorrowerZip'] = df['BorrowerZip'].astype('str')
# get the mean value across all data points
zipcode_data = df.groupby('BorrowerZip').aggregate(np.mean)
zipcode_data.reset_index(inplace = True)

# count number of companies grouped by zipcode
df['count'] = 1
temp = df.groupby('BorrowerZip').sum()
temp.reset_index(inplace = True)
temp = temp[['BorrowerZip', 'count']]
zipcode_data = pd.merge(zipcode_data, temp, on='BorrowerZip')
# drop count from org dataset
df.drop(['count'], axis = 1, inplace = True)


# Get geo data file path
geo_data_file = os.path.join('/content/drive/MyDrive/repos/team-74-capstone/data', 'nyc.json')
# load GeoJSON
with open(geo_data_file, 'r') as jsonFile:
    geo_data = json.load(jsonFile)
    
tmp = geo_data
# remove ZIP codes not in geo data
geozips = []

# print(list(zipcode_data['BorrowerZip'].unique()))
for i in range(len(tmp['features'])):
    if tmp['features'][i]['properties']['postalCode'] in list(zipcode_data['BorrowerZip'].unique()):
        geozips.append(tmp['features'][i])
        
# creating new JSON object
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips
print('hello',new_json)
# save updated JSON object
open("/content/drive/MyDrive/repos/team-74-capstone/data/cleaned_geodata.json", "w").write(json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': ')))

In [ ]:

def map_feature_by_zipcode(zipcode_data, col):
    """
    Generates a folium map of New York City
    :param zipcode_data: zipcode dataset
    :param col: feature to display
    :return: m
    """

    # read updated geo data
    king_geo = "/content/drive/MyDrive/repos/team-74-capstone/data/cleaned_geodata.json"

    # Initialize Folium Map with Seattle latitude and longitude
    m = folium.Map(location=[40.693943, -73.985880], zoom_start=9,
                   detect_retina=True, control_scale=False)

    # Create choropleth map
    m.choropleth(
        geo_data=king_geo,
        name='choropleth',
        data=zipcode_data,
        # col: feature of interest
        columns=['BorrowerZip', col],
        key_on='feature.properties.postalCode',
        fill_color='OrRd',
        fill_opacity=0.9,
        line_opacity=0.2,
        legend_name='companies ' + col
    )

    folium.LayerControl().add_to(m)

    # Save map based on feature of interest
    m.save(col + '.html')

    return m

In [ ]:
import folium
map_feature_by_zipcode(zipcode_data, 'count')

In [ ]:
# duplicateDFRow = df[df.duplicated(['BorrowerName'])]
# #duplicateDFRow = df[df.duplicated(['Name', 'Seasons'])]
# print(duplicateDFRow)

In [ ]:
# df.describe().apply(lambda s: s.apply('{0:.2f}'.format))

In [ ]:
# df.head(3)

In [ ]:
# df.corr()

In [ ]:
# !git status
